In [1]:
import keras
from keras import backend as K
import numpy as np
import io
import base64
from IPython.display import HTML
#import skvideo
#skvideo.setFFmpegPath("/usr/local/bin")
import skvideo.io
import cv2
import json

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# Project on Deep Reinforcement Learning and environment modifications

__Notations__: $E_p$ is the expectation under probability $p$.

# Context

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [2]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
        self.vis = 2
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

---
# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [8]:
class Environment(object):
    def __init__(self, grid_size=20, max_time=200, temperature=0.3, immob_penalty=0):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature 
        self.immob_penalty = immob_penalty
        self.vis = 2
        # this will impact the number of positive and negative rewards in the environment

        # board on which one plays
        self.board = np.zeros((grid_size,grid_size)) # possible reward in each location on the board
        self.position = np.zeros((grid_size,grid_size)) # the very position of the rat at time t
        self.malus_position = np.zeros((self.grid_size, self.grid_size)) # recording malus rewards on visited locations
        
        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        

    def draw(self,e):
        skvideo.io.vwrite('runs/' + str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0, 0] = 256
        b[self.big_bonus>0, 1] = 256
        b[self.board<0, 2] = 256
        b[self.x,self.y,:] = 256
        b[-4:,:,:]=0
        b[:,-4:,:]=0
        b[:4,:,:]=0
        b[:,:4,:]=0
        
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:] = b
        
        
    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:4, :] = -1
        self.position[:, 0:4] = -1
        self.position[-4:, :] = -1
        self.position[:, -4:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-5:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 4:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 5:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 4:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        ### Then before giving the reward, we give the environment the possibility to change and act differently
        self.modify_board() # Everything needed can be found in self.x, self.y, self.board, self.t, etc...
        
        reward = 0
        if train:
            reward += self.malus_position[self.x, self.y]
            reward += self.big_bonus[self.x, self.y]
            
        self.malus_position[self.x, self.y] = - self.immob_penalty
        self.big_bonus[self.x, self.y] = 0
        
        reward += self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1),
                                self.big_bonus.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[(self.x-self.vis):(self.x+1+self.vis),(self.y-self.vis):(self.y+1+self.vis),:]

        state_troncated = np.zeros((9,9,state.shape[2]))
        if self.vis == 4 :
            state_troncated = state
        else:
            state_troncated[(4-self.vis):(self.vis-4),(4-self.vis):(self.vis-4),:]=state
        
        return state_troncated, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
        self.vis = 2
        self.x = np.random.randint(5, self.grid_size-5, size=1)[0]
        self.y = np.random.randint(5, self.grid_size-5, size=1)[0]

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.big_bonus = np.zeros((self.grid_size, self.grid_size))
        
        island_size = self.grid_size - 8
        grid_bonus = 0.5*np.random.binomial(1, self.temperature, size=island_size**2)
        grid_bonus = grid_bonus.reshape(island_size, island_size)
        bonus = np.zeros((self.grid_size, self.grid_size))
        bonus[4:-4,4:-4] = grid_bonus

        grid_malus = -1.0*np.random.binomial(1, self.temperature, size=island_size**2)
        grid_malus = grid_malus.reshape(island_size, island_size)
        malus = np.zeros((self.grid_size, self.grid_size))
        malus[4:-4,4:-4] = grid_malus

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[:4, :] = -1
        self.position[:, :4] = -1
        self.position[-4:,:] = -1
        self.position[:,-4:] = -1
        self.board[self.x,self.y] = 0 # since we assume that we don't consider the first state in the reward,
        self.malus_position[self.x, self.y] = - self.immob_penalty # we can consider the original position as visited (or not ?)
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1),
                                self.big_bonus.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        state_troncated = np.zeros((9,9,state.shape[2]))
        if self.vis == 4 :
            state_troncated = state
        else:
            state_troncated[(4-self.vis):(self.vis-4),(4-self.vis):(self.vis-4),:]=state
       
        return state_troncated
    
    def modify_board(self):
        """This function modifies the board itself. It should use self.t and self.board"""
        pass
    
    def modify_visibility(self, win, lose):
        """This function modifies the visibility of the Agent. It should use self.vis"""
        pass
    
    def add_big_reward(self, win):
        """This function modifies the visibility of the Agent. It should use self.vis"""
        pass

Let's define now the hyper parameters :

In [9]:
# parameters
size = 19
T=500
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open('runs/'+name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

Let's now define the training loop and the test loop :

In [10]:
def train(agent, env,epoch, eps_decay=None, eps_step=None, freq_video=10, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    name = prefix
    name = name + '_explore_' if eps_step and eps_decay else name
    
    eps = agent.epsilon

    for e in range(epoch):
        
        if eps_step and eps_decay and e!=0 and e%eps_step==0:
            eps = eps*eps_decay
            agent.set_epsilon(eps)
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        
        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            
            state, reward, game_over = env.act(action, train=True)
            
            # Update the counters
            # We modify this interval because we only care about the true reward 
            # and not the penalty of coming back to visited places. This penalty aims at helping the algorithm
            # itself but shouldn't be considered in the win/lose counts
            win_before = win
            if reward > 0:
                win += reward
            elif reward != -env.immob_penalty:
                lose -= reward
            
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)
            
            env.modify_visibility(win, lose)
            
            if win_before < win :
                env.add_big_reward(win)
            
            agent.vis = env.vis
            
        # Save as a mp4
        if (e+1) % freq_video == 0:
            env.draw(name+str(e+1))

        # Update stats
        score += win-lose
        
        win_max = np.sum(np.where(env.to_draw[0, ::env.scale, ::env.scale, 0]==256, 1.0, 0))
        lose_max = np.sum(np.where(env.to_draw[0, ::env.scale, ::env.scale, 2]==256, 1.0, 0))

        print("Epoch {0:03d}/{1:03d} | Loss {2:.4f} | Win/lose count {3}({4:.1f}%)/{5}({6:.1f}%) ({7:.1f})"
              .format(e+1, epoch, loss, win, 100*win/win_max, lose, 100*lose/lose_max, win-lose))
        agent.save(name_weights=name+'_model.h5',name_model=name+'_model.json')
        
        
#------------------------------------------------------------------------------------------------

def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(1,epochs+1):
        # At each epoch, we restart to a fresh game and get the initial state
        
        state = env.reset()
        
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < -0.1:
                lose = lose - reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose
        
        win_max = np.sum(np.where(env.to_draw[0, ::env.scale, ::env.scale, 0]==256, 1.0, 0))
        lose_max = np.sum(np.where(env.to_draw[0, ::env.scale, ::env.scale, 2]==256, 1.0, 0))

        print("Win/lose count {}({:.1f}%)/{}({:.1f}%). Average score ({:.1f})"
              .format(win, 100*win/win_max, lose, 100*lose/lose_max, score/e))
    print('Final score: '+str(score/epochs))

## Random Agent

In [11]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()

    def learned_act(self, s):
        return np.random.randint(4)

Let's try it out ! *NB : This random agent cannot be trained so we test it directly*

In [12]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random1.mp4'))

Win/lose count 21.0(28.0%)/31.0(57.4%). Average score (-10.0)
Win/lose count 19.0(26.8%)/28.0(58.3%). Average score (-9.5)
Win/lose count 22.0(33.3%)/34.0(63.0%). Average score (-10.3)
Win/lose count 19.0(30.2%)/15.0(31.9%). Average score (-6.8)
Win/lose count 17.5(27.3%)/34.0(66.7%). Average score (-8.7)
Win/lose count 27.0(33.3%)/33.0(70.2%). Average score (-8.2)
Win/lose count 17.0(27.0%)/23.0(44.2%). Average score (-7.9)
Win/lose count 21.0(26.2%)/24.0(54.5%). Average score (-7.3)
Win/lose count 18.0(22.5%)/18.0(47.4%). Average score (-6.5)
Win/lose count 20.5(28.1%)/29.0(64.4%). Average score (-6.7)
Final score: -6.7


> A random agent is not great and performs really badly... Let's head to a more suitable approach when actually learning something by reinforcement. However, this agent will be useful to draw comparisons when modifying the environment.

***
## DQN

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
We will need a class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. A maximum memory size is required to avoid side effects.

In [ ]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        
        self.memory.append(m)
        if len(self.memory)>100:
            self.memory = self.memory[-100:]

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

In [ ]:
class DQN(Agent):
    def __init__(self, epsilon = 0.1, memory_size=100, batch_size = 16, n_state=4):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.8 # previous : 0.99
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        
        

    def learned_act(self, s):
        
        pred_rwd_per_action = self.model.predict(s.reshape(1,9,9,self.n_state))
        return np.argmax(pred_rwd_per_action)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
       
            
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 9, 9, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            sample = self.memory.random_access()
            
            input_states[i,:,:,:] = sample[0]
            
            target_q[i, sample[2]] = sample[3]
            
            if not game_over_:
                target_q[i, sample[2]] += self.discount*np.max(self.model.predict(sample[1].reshape(1,9,9,self.n_state)).flatten())
        
        target_q = np.clip(target_q, -30, 30)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self, name_weights='model.h5', name_model='model.json'):
        self.model.save_weights('runs/'+name_weights, overwrite=True)
        with open('runs/'+name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open('runs/'+name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights('runs/'+name_weights)
        model.compile("sgd", "mse")
        self.model = model


def my_custom_loss(y_true,y_pred):
    """
    Let's have a look at the issue when using the MSE loss. Take for example the situation in which :
       - Target = [   0, 0.8,    0,   0]
       - Output = [-0.3, 0.1, -1.5, 4.1]
    Then the MSE loss will also consider the distance between 0 and the conrresponding output value even though
    0 means that we don't have the information since the action wasn't considered. Thus the algorithm is mislead
    with a wrong information and it actually learns either to reproduce it's output, or to only output values
    close to 0.
    
    What could be much more meaningful would be to only consider the squared deviation between the target and
    the output but only for the information that we have.
    """
    norm = K.sum(y_true, axis=-1)+1e-6 # Get the value of the non-zero element from the target
    norm = K.reshape(norm, shape=(-1,1)) # Reshape it in order to concatenate it after
    norm = K.concatenate([norm, norm, norm, norm],axis=-1) # Concatenate in order to allow the next operation
    loss = K.sum(K.square(y_true * (y_true - y_pred) / norm), axis=-1) # Multiply the deviation by the target
    # thus the non selected actions won't be considered in the loss (and divide by the norm of the target not 
    # to impact the final loss)
    return loss

class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(32))
        model.add(Dense(64))
        model.add(Dense(4))
        
        loss = my_custom_loss #"mse"
        optimizer = Adam(lr=lr) #sgd(lr=lr, decay=1e-4, momentum=0.0), loss="mse")
        
        model.compile(optimizer=optimizer, loss=loss)
        self.model = model
        

In [ ]:
K.clear_session()
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(lr=.0001, epsilon = 0.3, memory_size=2000, batch_size = 64)
train(agent, env, epoch=20, prefix='fc_train')
HTML(display_videos('fc_train20.mp4'))

# we get impressively good results ! 
# It could be interesting to select the best set of parameters with a grid search

***
What if we were using a CNN instead ?

In [ ]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
        model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
        model.add(Flatten())
        model.add(Dense(16))
        model.add(Dense(4))
        
        loss = my_custom_loss #"mse"
        optimizer = Adam(lr=lr) #sgd(lr=lr, decay=1e-4, momentum=0.0), loss="mse")
        
        model.compile(optimizer=optimizer, loss=loss)
        self.model = model

In [ ]:
K.clear_session()
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(lr=.001, epsilon = 0.2, memory_size=2000, batch_size = 64)
train(agent, env, epoch=20, prefix='cnn_train')
HTML(display_videos('cnn_train20.mp4'))

### Comments :
# The loss represents the learning of the algorithm and the win/lose count represents somehow how well does the
# algorithm generalizes.

CNN seems to perform better that FC, but we see that they both struggle to find new cheese at some point... Exploration seems to be a limitation of our approach...

***
***
Compare both algorithms' performances :

In [ ]:
K.clear_session()
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_train_model.h5', name_model='cnn_train_model.json')

agent_fc = DQN_FC(lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_train_model.h5', name_model='fc_train_model.json')
print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix='cnn_test')
print('Test of the FC')
test(agent_fc, env, epochs_test, prefix='fc_test')

In [ ]:
HTML(display_videos('cnn_test10.mp4'))

In [ ]:
HTML(display_videos('fc_test10.mp4'))

> The local loss we introduced overcomes the issues related to the MSE one. In fact this is questionable since the MSE tends to make the network reproduce what it has done and penalizes bad moves (0 is better than negative rewards), but in our case, if we give a good exploration parameter we authorize the model to perform badly at first in order to get some cases to be able to predict future rewards. Once it has learned how to predict future rewards, it is thought to behave well but then would it get that bad rewards are worse than 0 ? This is questionable. In our case at least, it seems to work impressively well in very few epochs !!

***
The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. We use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not
---

In [ ]:
# Training
K.clear_session()
env_eplore = Environment(grid_size=size, max_time=T, temperature=0.3, immob_penalty=0.1)
agent = DQN_CNN(lr=.001, epsilon=0.3, memory_size=2000, batch_size=32, n_state=3)
train(agent, env, epoch=20, eps_decay=0.5, eps_step=3, prefix='cnn_train')
HTML(display_videos('cnn_train_explore_20.mp4'))

In [ ]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

> This approach works impressively well ! Indeed there is a local minimum in our original approach which needed to be avoided : since the model itself tries to be confident about its prediction somehow, one can think of the situation in which the mouse goes back and forth on 2 possible positions. Indeed, it will predict that the expected future reward is 0, which is true since it doesn't explore anymore and thus with a current reward of 0 and a future reward of 0, the model is able to predict very precisely its future and thus is in a local minimum. We need to avoid this situation by forcing an exploration. We need to put a high exploration parameter at first and then decrease it when the model has enough data to train on. The other part of the new approach deals with giving a minus reward when coming back on visited locations, which is great since the model is not able to predict the future reward precisely (infinite instead of 0).

---
---
---
# Adaptive Environment

To be tested :
- I) Modify the environment itself :
    - I-1) **Translation** of the board every T time step by a factor to be set as parameter
    - I-2) **Flip** the board based on a central/axial symmetry every T time step.
    - I-3) **Reset** to a random board at every T time step. (should break the algorithm and actually be pretty interesting considering the fact that the DQN approach with a CNN tries to be confident over its predictions... How can it be in this situation ?)
    - I-3 bis) **Reinitialize** the board (reset with same initialization) : it should be a sanity check
    - I-4) **Impact of the rat's actions** : based on what the rat eats, his vision can increase in dimension
    - I-5) **Impact of the rat's actions** : new rewards pop on the board based on the rat's actions
- II) Rewards follow a **distribution of probability** rather than being fixed : cheese ~ N(0.5, 1) | poison ~ N(-1, 0.5) for instance.
- III) **Adverse environnement** (environnement acts against the player). Implement and explore the Exp3 algorithm and compare it to the UCB1 one.

In [ ]:
class Adaptive_Environment(Environment):
    def __init__(self, *args, grid_size=13, max_time=200, temperature=0.3, transform_board=None,
                 transform_visibility=None, add_reward=None, **kwargs):
        super(Adaptive_Environment, self).__init__(*args,grid_size=grid_size, max_time=max_time, 
                                                   temperature=temperature,**kwargs)
        
        self.transform_board = transform_board
        self.transform_visibility = transform_visibility
        self.add_reward = add_reward

    def modify_board(self):
        """This function modifies the board itself. It should use self.t and self.board.
        To do so, we consider the transform_board function passed to the adaptative environment.
        Such a function should take as input the arguments of `self` and return the new board.
        """
        if self.transform_board is not None:
            self.board, self.malus_position = self.transform_board(infos=self)
        else:
            pass
    
    def modify_visibility(self,win,lose):
        """This function modifies the visibility area of the Agent. It should use self.vis.
        To do so, we consider the transform_visibility function passed to the adaptative environment.
        Such a function should take as input the arguments of `self` and return the new vis.
        """
        if self.transform_visibility is not None:
            self.vis = self.transform_visibility(infos=self,win=win,lose=lose)
        else:
            pass
    
    def add_big_reward(self,win):
        """This function modifies the visibility area of the Agent. It should use self.vis.
        To do so, we consider the transform_visibility function passed to the adaptative environment.
        Such a function should take as input the arguments of `self` and return the new vis.
        """
        if self.add_reward is not None:
            self.big_bonus = self.add_reward(infos=self,win=win)
        else:
            pass

### I - Modify the environment itself

#### 1 - Translation

In [ ]:
def board_translation(infos, time_step=2, amplitude=1):
    """Translate the board
    
    input :
    -----
        infos : parser
            An object containing informations about the environment (it is the self object from the class)
        time_step : int
            Frequency at which we want to modify the board.
        amplitude : int
            Number of columns (or rows, it doesn't matter given the symmetry of the game) to consider
            to shift the board.
            
    output :
    ------
        a new version of the board contained in the object self
    """
    board = infos.board
    malus_position = infos.malus_position
    if infos.t%time_step==0: # done after self.t += 1 so the case t=0 doesn't need to be considered
        # for instance with an amplitude of 2 : board[:, 2:] = board[:, :-2] | board[:, :2] = board[:, -2:]
        board[:, amplitude:], board[:, :amplitude] = board[:, :-amplitude], board[:, -amplitude:]
        malus_position[:, amplitude:], malus_position[:, :amplitude] = malus_position[:, :-amplitude], malus_position[:, -amplitude:]
    return board, malus_position

In [ ]:
K.clear_session()
##################################
epoch = 10
name = 'translate'
transform_fct = board_translation
##################################
mod_env = Adaptive_Environment(grid_size=size, max_time=500, temperature=0.3, immob_penalty=0.1,
                               transform_board=transform_fct)
agent = DQN_CNN(lr=.001, epsilon=0.3, memory_size=2000, batch_size=32, n_state=3)
train(agent, mod_env, epoch=epoch, eps_decay=0.5, eps_step=3, freq_video=1, prefix='{}_env'.format(name))
HTML(display_videos('{}_env_explore_{}.mp4'.format(name, epoch)))

#### 2 - Random Flip

In [ ]:
def board_flip(infos, time_step=10):
    """Translate the board
    
    input :
    -----
        infos : parser
            An object containing informations about the environment (it is the self object from the class)
        time_step : int
            Frequency at which we want to modify the board.
        amplitude : int
            Number of columns (or rows, it doesn't matter given the symmetry of the game) to consider
            to shift the board.
            
    output :
    ------
        a new version of the board contained in the object self
    """
    board = infos.board
    malus_position = infos.malus_position
    if infos.t%time_step==0: # done after self.t += 1 so the case t=0 doesn't need to be considered
        board = board[:, ::-1]
        malus_position = malus_position[:, ::-1]
    return board, malus_position

In [ ]:
K.clear_session()
##################################
epoch = 10
name = 'flip'
transform_fct = board_flip
##################################
mod_env = Adaptive_Environment(grid_size=size, max_time=500, temperature=0.3, immob_penalty=0.1,
                               transform_board=transform_fct)
agent = DQN_CNN(lr=.001, epsilon=0.3, memory_size=2000, batch_size=32, n_state=3)
train(agent, mod_env, epoch=epoch, eps_decay=0.5, eps_step=3, freq_video=1, prefix='{}_env'.format(name))
HTML(display_videos('{}_env_explore_{}.mp4'.format(name, epoch)))

In the case of time_step $\neq 1$ is the approach using batches selected randomly in the memory relevant ? I don't think so but then how should we modify it ?

#### 3 - Reset the board

In [ ]:
def board_reset(infos, time_step=10):
    """Translate the board
    
    input :
    -----
        infos : parser
            An object containing informations about the environment (it is the self object from the class)
        time_step : int
            Frequency at which we want to modify the board.
        amplitude : int
            Number of columns (or rows, it doesn't matter given the symmetry of the game) to consider
            to shift the board.
            
    output :
    ------
        a new version of the board contained in the object self
    """
    if infos.t%time_step==0: # done after self.t += 1 so the case t=0 doesn't need to be considered
        malus_position = np.zeros((infos.grid_size, infos.grid_size))

        bonus = 0.5*np.random.binomial(1, infos.temperature, size=infos.grid_size**2)
        bonus = bonus.reshape(infos.grid_size, infos.grid_size)

        malus = -1.0*np.random.binomial(1, infos.temperature, size=infos.grid_size**2)
        malus = malus.reshape(infos.grid_size, infos.grid_size)

        malus[bonus>0] = 0

        board = bonus + malus
    else:
        board = infos.board
        malus_position = infos.malus_position
    return board, malus_position

In [ ]:
K.clear_session()
##################################
epoch = 10
name = 'reset'
transform_fct = board_reset
##################################
mod_env = Adaptive_Environment(grid_size=size, max_time=500, temperature=0.3, immob_penalty=0.1,
                               transform_board=transform_fct)
agent = DQN_CNN(lr=.001, epsilon=0.3, memory_size=2000, batch_size=32, n_state=3)
train(agent, mod_env, epoch=epoch, eps_decay=0.5, eps_step=3, freq_video=1, prefix='{}_env'.format(name))
HTML(display_videos('{}_env_explore_{}.mp4'.format(name, epoch)))

**This last experiment is impressive !! 125.0+ | 11.0- it shows that the algorithm works impressively well at first and then struggle to find the final cheese. Here since the board is reset every 10 steps, the number of cheese is reset and the algorithm doesn't have to explore the whole board to get reward so it stays in place and eats every bonus in a certain global position. Great experiment !**

#### 3 (bis) - Reset the board (same initialisation)

In [ ]:
def board_reinit(infos, time_step=10):
    """Translate the board
    
    input :
    -----
        infos : parser
            An object containing informations about the environment (it is the self object from the class)
        time_step : int
            Frequency at which we want to modify the board.
        amplitude : int
            Number of columns (or rows, it doesn't matter given the symmetry of the game) to consider
            to shift the board.
            
    output :
    ------
        a new version of the board contained in the object self
    """
    if infos.t%time_step==0: # done after self.t += 1 so the case t=0 doesn't need to be considered
        malus_position = np.zeros((infos.grid_size, infos.grid_size))

        bonus = np.where(infos.to_draw[0, ::infos.scale, ::infos.scale, 0]==256, 0.5, 0)
        malus = np.where(infos.to_draw[0, ::infos.scale, ::infos.scale, 2]==256, -1.0, 0)
        
        board = bonus + malus
    else:
        board = infos.board
        malus_position = infos.malus_position
    return board, malus_position

In [ ]:
K.clear_session()
##################################
epoch = 10
name = 'reinit'
transform_fct = board_reinit
##################################
mod_env = Adaptive_Environment(grid_size=size, max_time=500, temperature=0.3, immob_penalty=0.1,
                               transform_board=transform_fct)
agent = DQN_CNN(lr=.001, epsilon=0.3, memory_size=2000, batch_size=32, n_state=3)
train(agent, mod_env, epoch=epoch, eps_decay=0.5, eps_step=3, freq_video=1, prefix='{}_env'.format(name))
HTML(display_videos('{}_env_explore_{}.mp4'.format(name, epoch)))

**4) Impact of the rat's actions : based on what the rat eats, his vision can increase/decrease in dimension**

In [ ]:
def visibility_changed(infos,win,lose):
    """Increases the visibility of the rat when it has eaten enough cheese
    
    input :
    -----
        infos : parser
            An object containing informations about the environment (it is the self object from the class)
        
            
    output :
    ------
        a new version of the visibility contained in the object self
    """
    vis = infos.vis
    
    if win%5==0 and vis < 4 and win>0:
        vis = vis +1
    elif lose%5==0 and vis>2 and lose>0 :
        vis = vis - 1
    return vis

In [ ]:
K.clear_session()
##################################
epoch = 10
name = 'visibility_changed'
transform_fct = visibility_changed
##################################
mod_env = Adaptive_Environment(grid_size=size, max_time=500, temperature=0.3, immob_penalty=0.1,
                               transform_visibility=transform_fct)
agent = DQN_CNN(lr=.001, epsilon=0.3, memory_size=2000, batch_size=32, n_state=3)
train(agent, mod_env, epoch=epoch, eps_decay=0.5, eps_step=3, freq_video=1, prefix='{}_env'.format(name))
HTML(display_videos('{}_env_explore_{}.mp4'.format(name, epoch)))

**5) Impact of the rat's actions : new rewards pop on the board based on the rat's actions**

In [ ]:
def reward_changed(infos, win, value=2):
    """Increases the visibility of the rat when it has eaten enough cheese
    
    input :
    -----
        infos : parser
            An object containing informations about the environment (it is the self object from the class)
        
            
    output :
    ------
        a new version of the visibility contained in the object self
    """
    big_bonus = infos.big_bonus
    
    if win%10==0 and win>0:
        x_cheese, y_cheese = np.random.randint(4,infos.grid_size-4,2)
        big_bonus[x_cheese, y_cheese]=value
    return big_bonus

In [ ]:
K.clear_session()
##################################
epoch = 10
name = 'big_cheese_added'
transform_fct = reward_changed
##################################
mod_env = Adaptive_Environment(grid_size=size, max_time=500, temperature=0.3, immob_penalty=0.1,
                               add_reward=transform_fct)
agent = DQN_CNN(lr=.001, epsilon=0.3, memory_size=2000, batch_size=32, n_state=4)
train(agent, mod_env, epoch=epoch, eps_decay=0.5, eps_step=3, freq_video=1, prefix='{}_env'.format(name))
HTML(display_videos('{}_env_explore_{}.mp4'.format(name, epoch)))

### II - Probability distribution

In [ ]:
def board_dist(infos):
    """Translate the board
    
    input :
    -----
        infos : parser
            An object containing informations about the environment (it is the self object from the class)
        time_step : int
            Frequency at which we want to modify the board.
        amplitude : int
            Number of columns (or rows, it doesn't matter given the symmetry of the game) to consider
            to shift the board.
            
    output :
    ------
        a new version of the board contained in the object self
    """
    board = infos.board
    malus_position = infos.malus_position
    x = infos.x
    y = infos.y
    
    if board[x,y]==0.5:
        board[x,y] = np.random.normal(0.5,1.0)
    elif board[x,y]==-1.0:
        board[x,y] = np.random.normal(-1.,0.5)
    
    ### Should we consider the malus_position as being a random variable as well ?
    #malus_position = infos.malus_position
    return board, malus_position

In [ ]:
K.clear_session()
##################################
epoch = 10
name = 'prob_dist'
transform_fct = board_dist
##################################
mod_env = Adaptive_Environment(grid_size=size, max_time=500, temperature=0.3, immob_penalty=0.1,
                               transform_board=transform_fct)
agent = DQN_CNN(lr=.001, epsilon=0.3, memory_size=2000, batch_size=32, n_state=3)
train(agent, mod_env, epoch=epoch, eps_decay=0.5, eps_step=3, freq_video=1, prefix='{}_env'.format(name))
HTML(display_videos('{}_env_explore_{}.mp4'.format(name, epoch)))